In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import random
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
import xgboost as xgb
from xgboost import XGBRegressor
# from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

#表示するテーブルの最大行と最大列を指定
pd.options.display.max_columns = 100
pd.options.display.max_rows = 999

seed = 42
# 乱数初期化
random.seed(seed)
np.random.seed(seed)

# データの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submit = pd.read_csv('sample_submit.csv')

# データ処理のためtrainとtestを結合
train_data = pd.concat([train, test])
print('trainサイズ:', len(train))
print('testサイズ:',  len(test))
print('train_dataサイズ:', len(train_data))

Duplicate key in file PosixPath('/Users/Y.Miura/anaconda3/lib/python3.11/site-packages/matplotlib/mpl-data/matplotlibrc'), line 263 ('font.family:  IPAexGothic')


trainサイズ: 3000
testサイズ: 2000
train_dataサイズ: 5000


In [2]:
# オブジェクトカラムをラベルエンコーディング
object_columns = train_data.select_dtypes(include=['object']).columns.tolist()
le = LabelEncoder()
for column in object_columns:
    train_data[column] = le.fit_transform(train_data[column])
    # train[column] = train[column].astype('cateogyr')
train_data[object_columns] = train_data[object_columns].astype('category')
train_data.dtypes

index                int64
Order                int64
MS SubClass          int64
MS Zoning         category
Lot Area             int64
Lot Shape         category
Land Contour      category
Lot Config        category
Neighborhood      category
Bldg Type         category
House Style       category
Overall Qual         int64
Overall Cond         int64
Year Built           int64
Year Remod/Add       int64
Roof Style        category
Exterior 1st      category
Exterior 2nd      category
Exter Qual        category
Foundation        category
BsmtFin SF 1       float64
Bsmt Unf SF        float64
Total Bsmt SF      float64
Heating QC        category
Central Air       category
Electrical        category
1st Flr SF           int64
2nd Flr SF           int64
Gr Liv Area          int64
Bsmt Full Bath     float64
Full Bath            int64
Half Bath            int64
Bedroom AbvGr        int64
Kitchen AbvGr        int64
Kitchen Qual      category
TotRms AbvGrd        int64
Fireplaces           int64
G

In [3]:
train

,index,Order,MS SubClass,MS Zoning,Lot Area,Lot Shape,Land Contour,Lot Config,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Exterior 1st,Exterior 2nd,Exter Qual,Foundation,BsmtFin SF 1,Bsmt Unf SF,Total Bsmt SF,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Gr Liv Area,Bsmt Full Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,200,2163,20,RL,9000,Reg,Lvl,Inside,NAmes,1Fam,1Story,5,5,1963,1950,Gable,VinylSd,VinylSd,TA,CBlock,554.725259,490.862838,620.339061,Ex,Y,SBrkr,941,0,2358,1.001999,2,0,3,1,TA,5,0,1.998003,381.014258,Y,0,0,7,2007,WD,Normal,158000
1,3832,1862,20,RL,9600,Reg,Lvl,Inside,CollgCr,1Fam,1Story,7,5,2006,2006,Gable,VinylSd,VinylSd,Gd,PConc,514.217911,1711.943709,1286.011022,Ex,Y,SBrkr,894,0,1040,0.002000,2,0,3,1,Gd,6,0,1.998003,384.114652,Y,0,0,11,2007,WD,Normal,145000
2,4927,754,20,RL,9600,Reg,Lvl,Inside,NridgHt,1Fam,1Story,9,5,2007,2007,Hip,VinylSd,VinylSd,Gd,PConc,1596.991227,511.217197,1341.741207,Ex,Y,SBrkr,864,0,864,1.001999,2,0,3,1,Gd,7,1,2.998015,625.140330,Y,0,0,5,2008,WD,Normal,125000
3,4088,641,80,RL,9600,Reg,Lvl,Inside,OldTown,1Fam,SLvl,6,6,1976,1950,Gable,VinylSd,Plywood,TA,CBlock,586.182353,469.685845,566.069609,Gd,Y,SBrkr,864,0,864,0.002000,1,0,3,1,TA,6,0,0.998002,193.156275,Y,0,0,7,2006,WD,Normal,135000
4,3644,754,20,RL,9600,Reg,Lvl,Inside,NoRidge,1Fam,1Story,7,5,1994,2002,Gable,VinylSd,VinylSd,Gd,PConc,1589.946047,484.337352,1258.013116,Ex,Y,SBrkr,832,0,1040,1.001998,2,0,3,1,Gd,7,1,2.998012,617.903766,Y,0,0,7,2007,WD,Normal,175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,4931,27,120,RL,1680,Reg,Lvl,Inside,NridgHt,TwnhsE,1Story,5,5,2004,2004,Gable,VinylSd,VinylSd,TA,PConc,620.110745,511.080630,1163.794125,Ex,Y,SBrkr,1337,0,1728,0.002000,2,0,3,1,TA,6,0,1.998001,416.886419,Y,0,0,5,2009,WD,Normal,130000
2996,3264,2203,160,RL,2665,Reg,Lvl,Inside,MeadowV,1Fam,2Story,5,5,1970,1970,Gable,CemntBd,MetalSd,TA,PConc,0.001057,480.872842,543.789909,Ex,Y,SBrkr,1040,0,1040,0.001999,1,1,3,1,TA,6,0,1.998004,371.721904,Y,0,0,5,2006,WD,Normal,140000
2997,1653,2447,20,RL,7200,Reg,Lvl,Inside,NAmes,1Fam,1Story,5,5,1963,1950,Gable,VinylSd,VinylSd,TA,CBlock,554.523591,488.283374,621.223013,Ex,Y,SBrkr,1098,0,2358,1.001999,2,0,3,1,TA,5,0,1.998003,379.306321,Y,0,0,7,2007,WD,Normal,158000
2998,2607,378,60,FV,9000,Reg,Lvl,Inside,Somerst,1Fam,2Story,6,5,2005,1950,Gable,VinylSd,VinylSd,Gd,PConc,0.000872,504.687298,599.525621,Ex,Y,SBrkr,728,0,1456,0.001999,2,1,3,1,TA,7,1,1.998002,371.138581,Y,0,0,7,2007,WD,Normal,158000


In [4]:
sample_submit.head()
test.head()

,index,Order,MS SubClass,MS Zoning,Lot Area,Lot Shape,Land Contour,Lot Config,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Exterior 1st,Exterior 2nd,Exter Qual,Foundation,BsmtFin SF 1,Bsmt Unf SF,Total Bsmt SF,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Gr Liv Area,Bsmt Full Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Mo Sold,Yr Sold,Sale Type,Sale Condition
0,398,414,60,RL,6120,IR1,Lvl,Inside,CollgCr,1Fam,2Story,8,5,1994,1950,Gable,VinylSd,VinylSd,TA,PConc,505.055258,197.338260,616.126989,Ex,Y,SBrkr,864,0,864,0.002000,2,0,3,1,Gd,7,1,2.998003,372.666174,Y,0,0,6,2006,WD,Normal
1,3833,2163,20,RL,9600,Reg,Lvl,Inside,Sawyer,1Fam,1Story,5,5,2008,1962,Gable,VinylSd,VinylSd,TA,PConc,1624.953706,494.176016,1285.118653,Ex,Y,SBrkr,1382,0,864,1.001999,2,0,3,1,TA,5,0,1.998014,637.974154,Y,0,0,3,2006,WD,Normal
2,4836,754,20,RL,9600,IR1,Lvl,Inside,NridgHt,1Fam,1Story,9,5,2006,1950,Gable,VinylSd,VinylSd,TA,PConc,0.001088,1813.402938,1294.560167,Ex,Y,SBrkr,912,0,864,0.002000,2,0,3,1,Ex,10,1,2.998008,605.913790,Y,0,0,5,2007,WD,Normal
3,4572,2284,80,RL,9600,IR1,Lvl,Inside,NAmes,1Fam,SLvl,5,5,1978,1950,Gable,Plywood,Plywood,TA,CBlock,567.457470,202.103285,598.983455,TA,Y,SBrkr,864,0,864,1.001999,1,0,3,1,TA,6,1,0.998020,200.755592,Y,0,0,7,2006,WD,Normal
4,636,2163,20,RL,7200,Reg,Lvl,Inside,NAmes,1Fam,1Story,5,5,1963,1950,Gable,VinylSd,VinylSd,TA,CBlock,555.696962,489.267758,622.300052,Ex,Y,SBrkr,941,0,848,1.001999,2,0,3,1,TA,5,0,1.998003,377.495159,Y,0,0,7,2007,WD,Normal


In [5]:
# #訓練データと検証データを分割
# X_train, X_val, y_train, y_val = train_test_split(
#     train_data.drop('SalePrice', axis=1).iloc[ : len(train)],
#     train_data['SalePrice'].iloc[ : len(train)],
#     shuffle=True, train_size=0.8,
#     random_state=seed)
# len(X_train)

In [ ]:
#交差検証
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

params = {'objective': 'regression',
          'metric': 'rmse',
          'num_leaves': 31,
          'learning_rate': 0.05,
          'n_estimators': 100}

cv = KFold(n_splits=3, random_state=seed, shuffle=True)

# RMSEスコアを格納するリスト
rmse_scores = []

# 交差検証
    
X = train_data.drop('SalePrice', axis=1).iloc[ : len(train)]
y = train_data['SalePrice'].iloc[ : len(train)]

for train_index, val_index in cv.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # LightGBM用データセット
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    
    lgb_model = lgb.train(params, lgb_train, num_boost_round=100,
                          valid_sets=[lgb_train, lgb_val],
                          callbacks=[lgb.early_stopping(stopping_rounds=10, 
                                verbose=True),lgb.log_evaluation(100)]
                          )

    # # 検証データで予測
    # y_pred = lgb.predict(X_val, num_iteration=lgb.best_iteration)
    # 
    # # RMSEを計算してリストに追加
    # rmse_score = np.sqrt(mean_squared_error(y_val, y_pred))
    # rmse_scores.append(rmse_score)
    
# スコアを表示
print('RMSE scores:', rmse_scores)
print('Average RMSE:', np.mean(rmse_scores))